In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction import text  
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import sqlite3
cnx = sqlite3.connect('db.sqlite3')



def is_spam(EmailText):
   
    df = pd.read_sql_query("SELECT * FROM application_data_set",cnx)
    
    train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
    X_train = train_set['EmailText']
    Y_train = train_set['Label']
    X_test = test_set['EmailText']
    Y_test = test_set['Label']
    
    
    vectorizer = text.CountVectorizer(stop_words="english")
    counts = vectorizer.fit_transform(X_train.values)
    spam_nb = MultinomialNB()
    spam_nb.fit(counts,Y_train.values)
    
    test_vect = vectorizer.transform(np.array([EmailText]))
    predict_test_vect = spam_nb.predict(test_vect)
    
    
    new_row = {'Label':predict_test_vect[0],'EmailText':EmailText,'id' :df.shape[0]+1}
    df = df.append(new_row,ignore_index=True)
    df.to_sql(name='application_data_set',con=cnx,if_exists = 'replace',index=False)
    
    return predict_test_vect[0] == 'spam'


In [ ]:
print(is_spam('Im really sorry, it wont happen again'))